<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/tutorials/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues

ESG-VARIABLES-PENALIZING
* **Pytorch Version**

# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)
    * [5.3. Initialize Environment](#4.3)
* [6.Implement DRL Algorithms](#5)
* [7.Backtesting Performance](#6)
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)
    * [7.3. Baseline Stats](#6.3)
    * [7.3. Compare to Stock Market Index](#6.4)

In [ ]:
pip install setuptools==66

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

In [ ]:
## install finrl library
!pip install wrds
!pip install swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-cr_igtwg
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-cr_igtwg
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to comm

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/usr/local/lib/python3.10/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [ ]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [ ]:
# Download and save the data in a pandas DataFrame:
df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2020-12-01',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2008-01-02,7.116786,7.152143,6.876786,5.914386,1079178800,AAPL,2
1,2008-01-02,46.599998,47.040001,46.259998,34.302551,7934400,AMGN,2
2,2008-01-02,52.090000,52.320000,50.790001,39.886971,8053700,AXP,2
3,2008-01-02,87.570000,87.839996,86.000000,63.481625,4303000,BA,2
4,2008-01-02,72.559998,72.669998,70.050003,46.068123,6337800,CAT,2


In [ ]:
df.shape

(94685, 8)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in highly volatile markets, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation in order to XXXXX.

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [ ]:
df.shape

(91056, 16)

In [ ]:
df.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2008-01-02,7.116786,7.152143,6.876786,5.914386,1079178800,AAPL,2,0.0,5.919615,5.911888,100.0,-66.666667,100.0,5.914386,5.914386
3252,2008-01-02,46.599998,47.040001,46.259998,34.302551,7934400,AMGN,2,0.0,5.919615,5.911888,100.0,-66.666667,100.0,34.302551,34.302551
6504,2008-01-02,52.090000,52.320000,50.790001,39.886971,8053700,AXP,2,0.0,5.919615,5.911888,100.0,-66.666667,100.0,39.886971,39.886971
9756,2008-01-02,87.570000,87.839996,86.000000,63.481625,4303000,BA,2,0.0,5.919615,5.911888,100.0,-66.666667,100.0,63.481625,63.481625
13008,2008-01-02,72.559998,72.669998,70.050003,46.068123,6337800,CAT,2,0.0,5.919615,5.911888,100.0,-66.666667,100.0,46.068123,46.068123


## Add covariance matrix as states

In [ ]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


In [ ]:
df.shape

(84000, 18)

In [ ]:
df.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,3.070357,3.133571,3.047857,2.590807,607541200,AAPL,2,-0.082824,3.101911,2.460843,42.254764,-80.468631,16.129793,2.756901,2.869310,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...
1,2008-12-31,57.110001,58.220001,57.060001,42.510155,6287200,AMGN,2,0.159269,43.391570,41.508011,51.060616,51.543691,10.432018,41.716138,41.254600,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...
2,2008-12-31,17.969999,18.750000,17.910000,14.690863,9625600,AXP,2,-0.942649,18.787692,12.756086,42.554836,-75.418965,25.776759,15.862965,17.749414,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...
3,2008-12-31,41.590000,43.049999,41.500000,32.005882,5443100,BA,2,-0.279800,32.174382,28.867837,47.440220,156.994540,5.366299,30.327214,32.389916,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...
4,2008-12-31,43.700001,45.099998,43.700001,29.909035,6277400,CAT,2,0.662263,30.655975,25.713892,51.205310,98.385993,26.331746,26.960317,26.691659,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [ ]:
train = data_split(df, '2009-01-01','2020-07-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)

## Environment for Portfolio Allocation


In [ ]:
train.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2009-01-02,3.067143,3.251429,3.041429,2.754725,746015200,AAPL,4,-0.070340,3.088352,2.458769,45.440194,-32.186088,2.140064,2.757751,2.870115,"[[0.001366152416301202, 0.00043393861067317563...",tic AAPL AMGN AXP ...
0,2009-01-02,58.590000,59.080002,57.750000,43.422924,6547900,AMGN,4,0.235884,43.539497,41.471973,52.756850,92.907818,0.814217,41.781161,41.344160,"[[0.001366152416301202, 0.00043393861067317563...",tic AAPL AMGN AXP ...
0,2009-01-02,18.570000,19.520000,18.400000,15.308586,10955700,AXP,4,-0.838333,18.602691,12.739929,43.957532,-42.828222,16.335101,15.861644,17.631675,"[[0.001366152416301202, 0.00043393861067317563...",tic AAPL AMGN AXP ...
0,2009-01-02,42.799999,45.560001,42.779999,33.941090,7010200,BA,4,-0.002009,32.948624,28.452134,50.822025,272.812669,20.494464,30.469479,32.344131,"[[0.001366152416301202, 0.00043393861067317563...",tic AAPL AMGN AXP ...
0,2009-01-02,44.910000,46.980000,44.709999,31.408848,7117200,CAT,4,0.841637,31.163168,25.692798,53.661269,129.572800,34.637448,27.199572,26.692247,"[[0.001366152416301202, 0.00043393861067317563...",tic AAPL AMGN AXP ...


In [ ]:
df["tic"].unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
       'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'VZ', 'WBA', 'WMT'],
      dtype=object)

In [ ]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


#File is only read here.
ESG_excel_file = pd.ExcelFile('DOW_ESG_curated.xlsx')

#Then info of sheets is just selected like this.
print(pd.read_excel(ESG_excel_file, 'IBM').columns)
print(pd.read_excel(ESG_excel_file, 'IBM').head())

Index(['Dates', 'ESG_SCORE', 'ENVIRONMENTAL_SCORE', 'SOCIAL_SCORE',
       'GOVERNANCE_SCORE'],
      dtype='object')
       Dates  ESG_SCORE  ENVIRONMENTAL_SCORE  SOCIAL_SCORE  GOVERNANCE_SCORE
0 2023-03-31        4.0                 6.08          3.07              7.37
1 2023-02-28        4.0                 6.08          3.07              7.37
2 2023-01-31        4.0                 6.08          3.07              7.37
3 2022-12-31        4.0                 6.08          3.07              7.37
4 2022-11-30        4.0                 6.08          3.07              7.37


In [ ]:
#We must include the ESG information in the dataframe to penalize it.
tics = ESG_excel_file.sheet_names
tics.remove('Worksheet')
tics.remove('Hoja1')

In [ ]:
ESG_dict = dict.fromkeys(tics, 0)
ESG_dict

{'DOW': 0,
 'AAPL': 0,
 'AMGN': 0,
 'AXP': 0,
 'BA': 0,
 'CAT': 0,
 'CRM': 0,
 'CSCO': 0,
 'CVX': 0,
 'DIS': 0,
 'GS': 0,
 'HD': 0,
 'HON': 0,
 'IBM': 0,
 'INTC': 0,
 'JNJ': 0,
 'JPM': 0,
 'KO': 0,
 'MCD': 0,
 'MMM': 0,
 'MRK': 0,
 'MSFT': 0,
 'NKE': 0,
 'PG': 0,
 'TRV': 0,
 'UNH': 0,
 'VZ': 0,
 'WBA': 0,
 'WMT': 0}

In [ ]:
for key in ESG_dict.keys():
  ESG_dict[key] = pd.read_excel(ESG_excel_file, key)
ESG_dict

{'DOW':          Dates  ESG_SCORE  ENVIRONMENTAL_SCORE  SOCIAL_SCORE  GOVERNANCE_SCORE
 0   2023-03-31       5.27                 4.63          4.43              7.42
 1   2023-02-28       5.27                 4.63          4.43              7.42
 2   2023-01-31       5.27                 4.63          4.43              7.42
 3   2022-12-31       5.27                 4.63          4.43              7.42
 4   2022-11-30       5.27                 4.63          4.43              7.42
 ..         ...        ...                  ...           ...               ...
 119 2013-04-30       4.63                 4.03          3.52              7.06
 120 2013-03-31       4.63                 4.03          3.52              7.06
 121 2013-02-28       4.63                 4.03          3.52              7.06
 122 2013-01-31       4.63                 4.03          3.52              7.06
 123 2012-12-31       4.63                 4.03          3.52              7.06
 
 [124 rows x 5 columns],
 'AAPL

In [ ]:
ESG_dict['IBM']

,Dates,ESG_SCORE,ENVIRONMENTAL_SCORE,SOCIAL_SCORE,GOVERNANCE_SCORE
0,2023-03-31,4.0,6.08,3.07,7.37
1,2023-02-28,4.0,6.08,3.07,7.37
2,2023-01-31,4.0,6.08,3.07,7.37
3,2022-12-31,4.0,6.08,3.07,7.37
4,2022-11-30,4.0,6.08,3.07,7.37
...,...,...,...,...,...
119,2013-04-30,3.1,3.13,0.84,7.52
120,2013-03-31,3.1,3.13,0.84,7.52
121,2013-02-28,3.1,3.13,0.84,7.52
122,2013-01-31,3.1,3.13,0.84,7.52


Just an utility function to find the closest date to a given date. Items is the list of dates and pivot the date to find the one that is most closed to.

In [ ]:
import time
import datetime

def str_to_timestamp(str_date):
  element = datetime.datetime.strptime(str_date,"%Y-%m-%d")
  return time.mktime(element.timetuple())

def nearest(items, pivot):
    return min(items, key=lambda x: abs(time.mktime(x.timetuple()) - str_to_timestamp(pivot)))

**ESG variables**

Only execute this cell if we want to consider ESG variables into the state space of the robot

In [ ]:
df["ESG_SCORE"] = 0
df["ENVIRONMENTAL_SCORE"] = 0
df["SOCIAL_SCORE"] = 0
df["GOVERNANCE_SCORE"] = 0
df.head()

print(len(df))
for instance_index in range(len(df)):
  instance_tic = df.iloc[[instance_index]]["tic"][instance_index]
  instance_date = df.iloc[[instance_index]]["date"][instance_index]
  operation_date = df.date.unique() #Retrieve the date for ESG values, we must find those values in the dictionary for every TIC.
  nearest_known_ESG_date = nearest(ESG_dict["DOW"]["Dates"], operation_date[0])
  df.iloc[instance_index, df.columns.get_loc("ESG_SCORE")] = float(ESG_dict[instance_tic][nearest_known_ESG_date==ESG_dict[instance_tic]["Dates"]]["ESG_SCORE"])
  df.iloc[instance_index, df.columns.get_loc("ENVIRONMENTAL_SCORE")] = float(ESG_dict[instance_tic][nearest_known_ESG_date==ESG_dict[instance_tic]["Dates"]]["ENVIRONMENTAL_SCORE"])
  df.iloc[instance_index, df.columns.get_loc("SOCIAL_SCORE")] = float(ESG_dict[instance_tic][nearest_known_ESG_date==ESG_dict[instance_tic]["Dates"]]["SOCIAL_SCORE"])
  df.iloc[instance_index, df.columns.get_loc("GOVERNANCE_SCORE")] = float(ESG_dict[instance_tic][nearest_known_ESG_date==ESG_dict[instance_tic]["Dates"]]["GOVERNANCE_SCORE"])
  if instance_index % 100 == 0:
    print(instance_index)


84000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18

Here is the definition of the environment.

In [ ]:
df

,date,open,high,low,close,volume,tic,day,macd,boll_ub,...,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,ESG_SCORE,ENVIRONMENTAL_SCORE,SOCIAL_SCORE,GOVERNANCE_SCORE
0,2008-12-31,3.070357,3.133571,3.047857,2.590807,607541200,AAPL,2,-0.082824,3.101911,...,-80.468631,16.129793,2.756901,2.869310,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...,4.563333,3.00,2.34,8.35
1,2008-12-31,57.110001,58.220001,57.060001,42.510155,6287200,AMGN,2,0.159269,43.391570,...,51.543691,10.432018,41.716138,41.254600,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...,3.120000,7.14,0.68,7.88
2,2008-12-31,17.969999,18.750000,17.910000,14.690863,9625600,AXP,2,-0.942649,18.787692,...,-75.418965,25.776759,15.862965,17.749414,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...,3.320000,3.83,1.67,6.71
3,2008-12-31,41.590000,43.049999,41.500000,32.005882,5443100,BA,2,-0.279800,32.174382,...,156.994540,5.366299,30.327214,32.389916,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...,3.110000,3.08,1.24,7.25
4,2008-12-31,43.700001,45.099998,43.700001,29.909035,6277400,CAT,2,0.662263,30.655975,...,98.385993,26.331746,26.960317,26.691659,"[[0.0013489706629158174, 0.0004284130806034819...",tic AAPL AMGN AXP ...,3.720000,2.61,1.91,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83995,2020-11-30,133.160004,134.000000,129.570007,122.698822,2684100,TRV,0,3.220512,133.353091,...,28.279922,8.999315,121.718069,113.728510,"[[0.0008489359055273687, 0.0004425212536417675...",tic AAPL AMGN AXP ...,1.650000,0.00,0.68,6.79
83996,2020-11-30,335.109985,337.940002,330.290009,325.150421,5265700,UNH,0,2.513450,355.410208,...,-1.881882,3.397972,323.730964,312.293370,"[[0.0008489359055273687, 0.0004425212536417675...",tic AAPL AMGN AXP ...,3.220000,1.34,1.67,7.54
83997,2020-11-30,60.320000,60.599998,60.189999,53.065155,21839100,VZ,0,0.490095,54.808123,...,60.120597,15.580037,51.810777,51.848694,"[[0.0008489359055273687, 0.0004425212536417675...",tic AAPL AMGN AXP ...,3.410000,2.51,2.12,8.21
83998,2020-11-30,39.009998,39.099998,37.889999,34.535732,13859300,WBA,0,0.404188,39.116836,...,11.612475,2.428551,34.357281,33.357611,"[[0.0008489359055273687, 0.0004425212536417675...",tic AAPL AMGN AXP ...,3.060000,0.00,1.56,8.28


In [ ]:
class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                ESG_dict,
                ESG_weight,
                ESG_combined,
                E_weight,
                S_weight,
                G_weight,
                apply_ESG_policy,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.ESG_dict = ESG_dict
        self.ESG_weight = ESG_weight
        self.portfolio_weight = 1.0 - self.ESG_weight
        self.ESG_combined = ESG_combined #Handles combined ESG index, not separate.
        self.apply_ESG_policy = apply_ESG_policy

        #Only used if ESG_combined are false.
        self.E_weight = E_weight
        self.S_weight = S_weight
        self.G_weight = G_weight

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            print(self.state)

            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)

            if(self.apply_ESG_policy):
              #calcualte portfolio return
              # individual stocks' return * weight ------ ADD HERE ESG PORTFOLIO? MORE RESTRICTIVE THAN IF IN AGENT
              operation_date = self.data.date.unique() #Retrieve the date for ESG values, we must find those values in the dictionary for every TIC.
              dow_esg_data = self.ESG_dict["DOW"]
              #Hay que calcular aqui los weights del ESG, referenciando por date y tic.
              nearest_known_ESG_date = nearest(dow_esg_data["Dates"], operation_date[0])
              #Extract the DOW ESG score for that date.
              dow_ESG_score = float(dow_esg_data[nearest_known_ESG_date==self.ESG_dict["DOW"]["Dates"]]["ESG_SCORE"])
              #Extract the Portfolio ESG Score for that date. Previous operation over all the dictionary.
              tics = [key for key in self.ESG_dict.keys() if key != 'DOW']
              ESG_scores_date = np.array([])
              for tic in tics:
                tic_esg_data = self.ESG_dict[tic]
                tic_ESG_score = tic_esg_data[nearest_known_ESG_date==self.ESG_dict[tic]["Dates"]]["ESG_SCORE"]
                ESG_scores_date = np.append(ESG_scores_date, tic_ESG_score)

              portfolio_ESG_score = np.sum(weights*ESG_scores_date)

              #Bonification if Dow_ESG<Portfolio_ESG_Score given by regulators in the portfolio return.
              #This makes the reward change and the new policy should benefit ESG companies.
              if dow_ESG_score >= portfolio_ESG_score:
                #ESG Regulation on return. Penalize if Dow_ESG>Portfolio_ESG_score in return due to regulations: ESG transaction cost.
                portfolio_return = portfolio_return - np.abs(portfolio_return)*((dow_ESG_score-portfolio_ESG_score)/dow_ESG_score)*self.ESG_weight
              else:
                #Grant per ESG-oriented portfolio on return.
                portfolio_return = portfolio_return + np.abs(portfolio_return)*((portfolio_ESG_score-dow_ESG_score)/(10.0-dow_ESG_score))*self.ESG_weight

            #...Weights tbc by investor´s preference
            # portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)

            #Aqui es donde hay que ponderar el ESG.
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 28, State Space: 28


In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "apply_ESG_policy": True,
    "ESG_dict": ESG_dict,
    "ESG_weight" : 0.5,
    "ESG_combined" : True,
    "E_weight" : 1.0/3.0,
    "S_weight" : 1.0/3.0,
    "G_weight" : 1.0/3.0
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
# initialize
agent_esg = DRLAgent(env = env_train)

### Model 1: **A2C**


In [ ]:
agent_esg = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c_esg = agent_esg.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [ ]:
trained_a2c_esg = agent_esg.train_model(model=model_a2c_esg,
                                tb_log_name='a2c',
                                total_timesteps=50000)

Streaming output truncated to the last 5000 lines.
  4.45647833e+01 1.75351595e+01]
 [4.04061718e+01 1.55738003e+02 9.12318492e+01 ... 4.08077702e+01
  6.14694336e+01 9.29580345e+01]
 [4.05909504e+01 1.51605285e+02 9.04241796e+01 ... 3.97113638e+01
  6.02007785e+01 9.02984525e+01]]
[[1.42914770e-04 3.91778616e-05 5.13808524e-05 ... 7.87053041e-06
  3.80858795e-05 2.28201428e-05]
 [3.91778616e-05 1.35160743e-04 4.07803954e-05 ... 2.43983572e-05
  5.60934191e-05 3.79664097e-05]
 [5.13808524e-05 4.07803954e-05 1.04609758e-04 ... 3.42672686e-05
  3.04987733e-05 2.20463429e-05]
 ...
 [3.84159509e+01 1.83442522e+01 3.99362914e+01 ... 3.15676193e+01
  4.34158133e+01 1.75351595e+01]
 [4.03452803e+01 1.55727272e+02 9.10880391e+01 ... 4.07371291e+01
  6.13464128e+01 9.29625610e+01]
 [4.05594776e+01 1.51688515e+02 9.04438148e+01 ... 3.97943672e+01
  6.01715336e+01 9.04333276e+01]]
[[1.42993234e-04 3.92235116e-05 5.11512963e-05 ... 8.12333530e-06
  3.78115062e-05 2.34459934e-05]
 [3.92235116e-05 1

In [ ]:
trained_a2c.save('/content/trained_models/trained_a2c.zip')

### Model 2: **PPO**


In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=80000)

[[1.35202362e-03 4.29471564e-04 1.00226386e-03 ... 4.97252212e-04
  4.88315205e-04 3.47737391e-04]
 [4.29471564e-04 7.50375800e-04 5.61090694e-04 ... 3.64908912e-04
  3.49639469e-04 2.93433325e-04]
 [1.00226386e-03 5.61090694e-04 2.11222012e-03 ... 8.30237330e-04
  7.32800124e-04 5.83658075e-04]
 ...
 [1.49239343e+01 5.04498202e+00 8.21537066e+00 ... 5.55854258e+00
  2.53261313e+01 1.73407485e+01]
 [2.76613863e+00 4.19286284e+01 1.58935868e+01 ... 1.49641512e+01
  1.70666503e+01 3.97366220e+01]
 [2.87253821e+00 4.14365412e+01 1.75370362e+01 ... 1.41258310e+01
  1.66548378e+01 3.90964374e+01]]
[[1.35234717e-03 4.31371902e-04 9.99170869e-04 ... 4.98984824e-04
  4.89632119e-04 3.49108180e-04]
 [4.31371902e-04 7.52008076e-04 5.55318449e-04 ... 3.65501759e-04
  3.48937742e-04 2.93549852e-04]
 [9.99170869e-04 5.55318449e-04 2.12566727e-03 ... 8.26139107e-04
  7.31183954e-04 5.80874185e-04]
 ...
 [1.77529813e+01 9.95234250e+00 3.21770310e+00 ... 9.43319654e+00
  2.85284187e+01 1.63075922e+01]

KeyboardInterrupt: ignored

In [ ]:
trained_ppo.save('/content/trained_models/trained_ppo.zip')

### Model 3: **DDPG**


In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4981447.8010044675
Sharpe:  0.8613296544165371
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 174       |
|    time_elapsed    | 66        |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -8.42e+07 |
|    critic_loss     | 5.81e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 8679      |
|    reward          | 5267437.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asse

In [ ]:
trained_ddpg.save('/content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

In [ ]:
trained_sac.save('/content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

In [ ]:
trained_td3.save('/content/trained_models/trained_td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [ ]:
trade = data_split(df,'2020-07-01', '2021-10-31')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)


In [ ]:
trade.shape

(2968, 22)

In [ ]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c_esg,
                        environment = e_trade_gym)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


[[6.52333270e-04 4.04412152e-04 6.58568541e-04 ... 2.53302177e-04
  4.10541204e-04 2.80110041e-04]
 [4.04412152e-04 5.44246092e-04 4.16447842e-04 ... 2.15452821e-04
  3.21118943e-04 2.57884416e-04]
 [6.58568541e-04 4.16447842e-04 1.35272359e-03 ... 3.03774239e-04
  5.09828012e-04 2.30003796e-04]
 ...
 [3.33356085e+01 5.16695845e+01 2.04374377e+00 ... 6.78668472e+00
  9.84903163e+00 3.26836719e+00]
 [8.38027463e+01 2.08862822e+02 9.59542274e+01 ... 4.80059466e+01
  3.78136504e+01 1.15553415e+02]
 [7.76873227e+01 2.09751314e+02 8.94866507e+01 ... 4.83650064e+01
  3.75094963e+01 1.17695707e+02]]
[[6.54581882e-04 4.03293269e-04 6.60838888e-04 ... 2.54126298e-04
  4.13286982e-04 2.79757309e-04]
 [4.03293269e-04 5.43619754e-04 4.15341212e-04 ... 2.15338490e-04
  3.20085111e-04 2.57783832e-04]
 [6.60838888e-04 4.15341212e-04 1.35501543e-03 ... 3.04600559e-04
  5.12593848e-04 2.29653038e-04]
 ...
 [3.90433507e+01 5.46445633e+01 5.98449794e-02 ... 6.47896834e+00
  3.80294498e+00 1.53950851e+00]

In [ ]:
df_daily_return.head()

,date,daily_return
0,2020-07-01,0.000000
1,2020-07-02,0.005160
2,2020-07-06,0.015392
3,2020-07-07,-0.014181
4,2020-07-08,0.005621


In [ ]:
df_daily_return.to_csv('df_daily_return_sin_esg.csv')

In [ ]:
df_actions.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714
2020-07-02,0.026130,0.071029,0.063537,0.026130,0.026130,0.026130,0.026130,0.029726,0.049668,0.043843,...,0.026130,0.026440,0.026130,0.026130,0.026130,0.026130,0.071029,0.071029,0.026130,0.033267
2020-07-06,0.026130,0.071029,0.063537,0.026130,0.026130,0.026130,0.026130,0.029726,0.049668,0.043843,...,0.026130,0.026440,0.026130,0.026130,0.026130,0.026130,0.071029,0.071029,0.026130,0.033267
2020-07-07,0.026130,0.071029,0.063537,0.026130,0.026130,0.026130,0.026130,0.029726,0.049668,0.043843,...,0.026130,0.026440,0.026130,0.026130,0.026130,0.026130,0.071029,0.071029,0.026130,0.033267
2020-07-08,0.026313,0.071525,0.062876,0.026313,0.026313,0.026313,0.026313,0.030078,0.049203,0.043159,...,0.026313,0.026734,0.026313,0.026313,0.026313,0.026313,0.071525,0.071525,0.026313,0.033763


In [ ]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func( returns=DRL_strat,
                              factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [ ]:
print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


Annual return          0.399769
Cumulative returns     0.151957
Annual volatility      0.183134
Sharpe ratio           1.928264
Calmar ratio           4.473839
Stability              0.530463
Max drawdown          -0.089357
Omega ratio            1.367914
Sortino ratio          2.969039
Skew                  -0.066524
Kurtosis               0.703152
Tail ratio             1.087166
Daily value at risk   -0.021671
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (105, 8)
Annual return          0.434549
Cumulative returns     0.162246
Annual volatility      0.178863
Sharpe ratio           2.127074
Calmar ratio           4.865749
Stability              0.551107
Max drawdown          -0.089308
Omega ratio            1.404285
Sortino ratio          3.173748
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.984517
Daily value at risk   -0.021025
dtype: float64


/usr/local/lib/python3.10/dist-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
pip install empyrical==0.3.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 16.6 MB/s eta 0:00:00
  Created wheel for empyrical: filename=empyrical-0.3.4-py3-none-any.whl size=27182 sha256=5a110695a77a75068b4f9743a893e4585605672dfbb52a8f423d6c9544133563
  Stored in directory: /root/.cache/pip/wheels/92/99/6a/89c359a6a9a86aa37d1d72db48b703f01f92d35ff31db5d193
Successfully built empyrical
  Attempting uninstall: empyrical
    Found existing installation: empyrical 0.5.5
    Uninstalling empyrical-0.5.5:
      Successfully uninstalled empyrical-0.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyfolio 0.9.2 requires empyrical>=0.5.0, but you have empyrical 0.3.

In [ ]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^DJI', start=df_daily_return.loc[0,'date'], end='2021-11-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (337, 8)


/usr/local/lib/python3.10/dist-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/usr/local/lib/python3.10/dist-packages/pyfolio/plotting.py:648: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


Start date,2020-07-01
End date,2020-11-30
Total months,5
,Backtest
Annual return,40.5%
Cumulative returns,15.4%
Annual volatility,17.6%
Sharpe ratio,2.02
Calmar ratio,4.74
Stability,0.59
Max drawdown,-8.6%


AttributeError: ignored

## Min-Variance Portfolio Allocation

In [ ]:
%pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [ ]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [ ]:
df.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,3.070357,3.133571,3.047857,2.590807,607541200,AAPL,2,-0.082824,3.101910,2.460843,42.254753,-80.468629,16.129793,2.756901,2.869311,"[[0.0013489689763092982, 0.0004284122097917537...",tic AAPL AMGN AXP ...
1,2008-12-31,57.110001,58.220001,57.060001,42.911251,6287200,AMGN,2,0.160772,43.800980,41.899652,51.060623,51.555769,10.432018,42.109742,41.643846,"[[0.0013489689763092982, 0.0004284122097917537...",tic AAPL AMGN AXP ...
2,2008-12-31,17.969999,18.750000,17.910000,14.690864,9625600,AXP,2,-0.942648,18.787691,12.756087,42.554850,-75.418955,25.776759,15.862964,17.749413,"[[0.0013489689763092982, 0.0004284122097917537...",tic AAPL AMGN AXP ...
3,2008-12-31,41.590000,43.049999,41.500000,32.005878,5443100,BA,2,-0.279801,32.174381,28.867832,47.440220,156.994497,5.366299,30.327212,32.389914,"[[0.0013489689763092982, 0.0004284122097917537...",tic AAPL AMGN AXP ...
4,2008-12-31,43.700001,45.099998,43.700001,29.909040,6277400,CAT,2,0.662263,30.655970,25.713891,51.205320,98.386062,26.331746,26.960314,26.691658,"[[0.0013489689763092982, 0.0004284122097917537...",tic AAPL AMGN AXP ...


In [ ]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

portfolio=portfolio.T
portfolio.columns = ['account_value']

In [ ]:
portfolio.head()

,account_value
2020-07-01,1000000
2020-07-02,1005253.9082
2020-07-06,1014948.858243
2020-07-07,1014218.861182
2020-07-08,1012644.11547


In [ ]:
a2c_cumpod_esg =(df_daily_return.daily_return+1).cumprod()-1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1

In [ ]:
dji_cumpod =(baseline_returns+1).cumprod()-1

## Plotly: DRL, Min-Variance, DJIA

In [ ]:
%pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [ ]:
time_ind = pd.Series(df_daily_return.date)

In [ ]:
trace0_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod, mode = 'lines', name = 'A2C (Portfolio Allocation)')

trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')
trace3_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod_esg, mode = 'lines', name = 'ESG-A2C (Portfolio Allocation)')
#trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
#trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
#trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

#trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

#trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
#trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

In [ ]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)

fig.add_trace(trace3_portfolio)

fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2

    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date',
       'tick0': time_ind[0],
        'tickmode': 'linear',
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()